<a href="https://colab.research.google.com/github/pkug28/shot_data_calculator/blob/main/Shot_Data_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
import io
import warnings

In [ ]:
warnings.filterwarnings('ignore')

# Import file here
data = files.upload()

Saving shots_data.csv to shots_data.csv


In [ ]:
# Convert to Pandas Dataframe
def convert_to_df(data: any, file: str):
  df = pd.read_csv(io.BytesIO(data[file]))
  return df

In [ ]:
shots = convert_to_df(data, 'shots_data.csv')

In [ ]:
# Function to convert types and add Linear Distance column
def clean_df(df):
  # Convert X and Y to double types
  df['x'].astype(np.double)
  df['y'].astype(np.double)
  # Calculate linear distance and add new column
  df['LinearDistance']=(np.sqrt((df['x']**2)+(df['y']**2))).astype(np.double)
  return df

In [ ]:
shots = clean_df(shots)
shots

,team,x,y,fgmade,LinearDistance
0,Team A,-5.0,26.4,0,26.869313
1,Team A,-0.8,1.2,1,1.442221
2,Team A,-13.9,9.5,1,16.836270
3,Team A,-5.4,26.2,0,26.750701
4,Team A,-4.9,14.5,1,15.305555
...,...,...,...,...,...
499,Team B,-0.5,2.2,0,2.256103
500,Team B,3.5,7.6,0,8.367198
501,Team B,0.2,8.8,0,8.802272
502,Team B,-0.6,0.7,1,0.921954


In [ ]:
# Return Unique Team Names
def team_names(df):
  teams = []
  for name in df['team'].unique():
    name = name.replace(" ", "_")
    teams.append(name)
  return teams

# Calculate Total Shot Attempts
def shot_attempts(df, team: str) -> int:
  attempts = df[df['team']==team]['team'].count().astype(int)
  return attempts

# Calculate shot distribution
def shot_distribution(attempts: int, total_attempts: int) -> float:
  distribution = attempts/total_attempts
  return distribution

# Calculate 2 Point Effective FG %
def efg(shot_type_makes: int, three_point_makes: int, shot_type_attempts: int) -> float:
  efg = ((shot_type_makes+(0.5*(three_point_makes)))/shot_type_attempts)
  return efg

In [ ]:
# Calculate 2 Point Attempts
def two_point_attempts(df, team: str) -> int:
  attempts = df[df['team']==team][df['x']<=22][df['x']>=-22][shots['LinearDistance']<=23.75]['team'].count().astype(int)
  return attempts

# Calculate 2 Point Makes
def two_point_makes(df, team: str) -> int:
  makes = df[df['team']==team][df['x']<=22][df['x']>=-22][shots['LinearDistance']<=23.75][df['fgmade']==1]['team'].count().astype(int)
  return makes

# Calculate 2 Point Percentage
def two_point_percentage(attempts: int, makes: int, team: str) -> float:
  percentage = makes/attempts*100
  return percentage

In [ ]:
# Calculate 3 Point Attempts from the Corners
def three_point_corner_attempts(df, team: str) -> int:
  attempts_right = df[df['team']==team][df['y']<=7.8][df['x']>=22]['team'].count().astype(int)
  attempts_left = df[df['team']==team][df['y']<=7.8][df['x']<=-22]['team'].count().astype(int)
  attempts = attempts_right + attempts_left
  return attempts

# Calculate 3 Point Makes from the Corners
def three_point_corner_makes(df ,team: str) -> int:
  makes_right = df[df['team']==team][df['y']<=7.8][df['x']>=22][df['fgmade']==1]['team'].count().astype(int)
  makes_left = df[df['team']==team][df['y']<=7.8][df['x']<=-22][df['fgmade']==1]['team'].count().astype(int)
  makes = makes_right + makes_left
  return makes

# Calculate 3 Point Percentage from the Corners
def three_point_corner_percentage(attempts: int, makes: int, team: str) -> int:
  percentage = makes/attempts*100
  return percentage


In [ ]:
# Calculate 3 Point Attempts above the break
def three_point_above_break_attempts(df, team: str) -> int:
  attempts = df[df['team']==team][df['y']>7.8][df['LinearDistance']>23.75]['team'].count().astype(int)
  return attempts

# Calculate 3 Point Makes above the break
def three_point_above_break_makes(df, team: str) -> int:
  makes = df[df['team']==team][df['y']>7.8][df['LinearDistance']>23.75][df['fgmade']==1]['team'].count().astype(int)
  return makes

# Calculate 3 Point Percentage above the break
def three_point_above_break_percentage(attempts: int, makes: int, team: str) -> int:
  percentage = makes/attempts*100
  return percentage

In [ ]:
teams = team_names(shots)
teams

['Team_A', 'Team_B']

In [ ]:
class team_obj:
  team_name: str
  shot_attempts: int
  two_point_attempts: int
  two_point_made: int
  two_point_distribution: float
  corner_attempts: int
  corner_made: int
  corner_distribution: float
  above_break_attempts: int
  above_break_made: int
  above_break_distribution: float
  three_point_attempts: int
  three_point_made: int
  two_point_efg: float
  corner_efg: float
  above_break_efg: float

  def __str__(self):
    return f"{self.team_name}"


In [ ]:
def create_team_objects(df, teams: list) -> list:
  team_objects = []
  for name in teams:
    team = team_obj()
    team.team_name = name.replace("_", " ")
    team.shot_attempts = shot_attempts(df, team.team_name)
    team.two_point_attempts = two_point_attempts(df, team.team_name)
    team.two_point_made = two_point_makes(df, team.team_name)
    team.corner_attempts = three_point_corner_attempts(df, team.team_name)
    team.corner_made = three_point_corner_makes(df, team.team_name)
    team.above_break_attempts = three_point_above_break_attempts(df, team.team_name)
    team.above_break_made = three_point_above_break_makes(df, team.team_name)
    team.three_point_attempts = team.corner_attempts + team.above_break_attempts
    team.three_point_made = team.corner_made + team.above_break_made
    team.two_point_distribution = shot_distribution(team.two_point_attempts, team.shot_attempts)
    team.corner_distribution = shot_distribution(team.corner_attempts, team.shot_attempts)
    team.above_break_distribution = shot_distribution(team.above_break_attempts, team.shot_attempts)
    team.two_point_efg = efg(team.two_point_made, team.three_point_made, team.two_point_attempts)
    team.corner_efg = efg(team.corner_made, team.three_point_made, team.corner_attempts)
    team.above_break_efg = efg(team.above_break_made, team.three_point_made, team.above_break_attempts)
    team_objects.append(team)
  return team_objects

In [ ]:
team_objects = create_team_objects(shots, teams)
print(team_objects[1].above_break_efg)

0.5403225806451613


In [ ]:
def data_to_df(objects: list, teams: list) -> pd.DataFrame:
  objs = []
  for obj in objects:
    obj_list = []
    obj_list.append(obj.two_point_distribution.round(3))
    obj_list.append(obj.two_point_efg.round(3))
    obj_list.append(obj.corner_distribution.round(3))
    obj_list.append(obj.corner_efg.round(3))
    obj_list.append(obj.above_break_distribution.round(3))
    obj_list.append(obj.above_break_efg.round(3))
    objs.append(obj_list)
  df = pd.DataFrame(np.array(objs),
                    columns=['Two Point Distribution', 'Two Point eFG', 'Corner Three Distribution', 'Corner Three eFG',
                                  'Above Break Three Distribution', 'Above Break Three eFG'],
                    index=teams)
  return df


In [ ]:
df_calcs = data_to_df(team_objects, teams)
df_calcs

,Two Point Distribution,Two Point eFG,Corner Three Distribution,Corner Three eFG,Above Break Three Distribution,Above Break Three eFG
Team_A,0.693,0.567,0.064,1.333,0.243,0.529
Team_B,0.674,0.526,0.054,1.375,0.277,0.540
